# Export data to CSVs for regressions

## Connect to Postgres database

In [ ]:
import pandas as pd
from dotenv import load_dotenv
load_dotenv()

from sqlalchemy import create_engine, inspect
import os

# build URL from the locally‑forwarded port
user     = os.getenv("DB_USER")
pw       = os.getenv("DB_PASSWORD")
host     = os.getenv("DB_HOST")
port     = os.getenv("DB_PORT")
db       = os.getenv("DB_NAME")
engine   = create_engine(f"postgresql://{user}:{pw}@{host}:{port}/{db}")

In [ ]:
ilogs = pd.read_sql("SELECT * FROM interaction_logs;", engine)
ilogs.head()

In [ ]:
surveys = pd.read_sql("SELECT * FROM survey_responses;", engine)
surveys.head()

In [ ]:
snapshots = pd.read_sql("SELECT * FROM text_snapshots;", engine)
snapshots.head()

## Get list of accepted participants

In [ ]:
pid_list = []

with open("pid_accepted.txt", "r") as fle:
    for line in fle:
        pid_list.append(line.strip())
        
len(pid_list), pid_list[0]

## Export pre-survey data

In [ ]:
filtered = surveys.loc[
    (surveys["participant_id"].isin(pid_list)) &
    (surveys["survey_type"] == "pre")
]

filtered = filtered.drop_duplicates(subset="participant_id", keep="last")

print(len(filtered), len(pid_list))

filtered.head()

In [ ]:
import json

def enrich_response(row):
    resp = row["responses"]
    if isinstance(resp, str):
        try:
            resp = json.loads(resp)
        except:
            resp = {}

    enriched = {
        **resp,  # unpack original keys
        "participant_id": row["participant_id"],
        "prompt_id": row["prompt_id"], 
        "condition": row["condition"]
    }
    return enriched

filtered["responses_enriched"] = filtered.apply(enrich_response, axis=1)
filtered

In [ ]:
responses = filtered["responses_enriched"].to_list()

presurvey = pd.DataFrame(responses)
presurvey.head()

In [ ]:
presurvey.to_csv('csv_exports/presurvey.csv', index=False)

## Export post-survey data

In [ ]:
filtered = surveys.loc[
    (surveys["participant_id"].isin(pid_list)) &
    (surveys["survey_type"] == "post")
]

filtered = filtered.drop_duplicates(subset="participant_id", keep="last")

print(len(filtered), len(pid_list))

filtered["responses_enriched"] = filtered.apply(enrich_response, axis=1)
filtered.head()

In [ ]:
responses = filtered["responses_enriched"].to_list()

postsurvey = pd.DataFrame(responses)
postsurvey.head()

In [ ]:
postsurvey.to_csv('csv_exports/postsurvey.csv', index=False)

## Export behavioral data

- time in each stage
- keystrokes in each stage
- number of AI support requests
- edit distance btwn draft and revision

### Test edit distance

In [ ]:
!pip install editdistance

In [ ]:
import editdistance
editdistance.eval('banana', 'bahama')

In [ ]:
def get_ai_draft(pid):
    filtered = ilogs.loc[ilogs["participant_id"] == pid]
    
    phrase = "Please write a complete draft essay based on this outline and prompt"
    matches = filtered[filtered["event_data"].apply(lambda d: isinstance(d, dict) and phrase in d.get("prompt", ""))]
    
    if len(matches) < 1:
        print(f"error [{pid}]: no ai draft found")
        return ""
    
    if len(matches) > 1:
        print(f"warning [{pid}]: more than one ai draft found")
        matches = matches.drop_duplicates(subset="participant_id", keep="last")
    
    return matches.iloc[0]["event_data"]["response"]

def get_essay(pid, stage):
    filtered = snapshots.loc[
        (snapshots["participant_id"] == pid) & 
        (snapshots["type"] == "final") &
        (snapshots["stage"] == stage)
    ]
    # if condition 3, need to pull draft from api request
    if len(filtered) == 0:
        condition = surveys.loc[surveys["participant_id"] == pid].iloc[0]["condition"]
        if condition == "3":
            return get_ai_draft(pid)
        else:
            print(f"error [{pid}]: no final submission [{stage}]")
            return "error"
    if len(filtered) > 1:
        print(f"warning [{pid}]: more than one final submission [{stage}]")
        filtered = filtered.drop_duplicates(subset="participant_id", keep="last")
    
    return filtered.iloc[0]["text_content"]


draft = get_essay(pid_list[0], "draft")
revision = get_essay(pid_list[0], "revision")
print("edit distance:", editdistance.eval(draft, revision))
print("------------------")
print(draft)
print("------------------")
print(revision)

### Prep functions

In [ ]:
def get_num_keystrokes(pid, stage):
    filtered = ilogs.loc[
        (ilogs["participant_id"] == pid) &
        (ilogs["stage"].str.lower() == stage)
    ]
    count = filtered['event_type'].str.contains('keystroke').sum()
    return count

def get_time_on_task(pid, stage):
    # return in minutes
    filtered = snapshots.loc[
        (snapshots["participant_id"] == pid) & 
        (snapshots["type"] == "final") &
        (snapshots["stage"] == stage)
    ]
    
    if len(filtered) > 1:
        print(f"warning [{pid}]: more than one stage [{stage}]")
        
    if len(filtered) == 0:
        return "null"
    
    return filtered.iloc[0]["time_from_stage_start"]/60  

def get_keystroke_events(pid, keyword):
    filtered = ilogs.loc[
        (ilogs["participant_id"] == pid) & 
        (ilogs["event_type"].str.contains(keyword))
    ]
    return len(filtered)

def get_api_requests(pid):
    filtered = ilogs.loc[ilogs["participant_id"] == pid]
    count = filtered['event_type'].str.contains('api_call').sum()
    return count

def get_edit_distance(pid):
    draft = get_essay(pid, "draft")
    revision = get_essay(pid, "revision")
    return editdistance.eval(draft, revision)

def get_condition(pid):
    filtered = surveys.loc[surveys["participant_id"] == pid]
    return filtered.iloc[0]["condition"]

def get_prompt_id(pid):
    filtered = surveys.loc[surveys["participant_id"] == pid]
    return filtered.iloc[0]["prompt_id"]

### Build new dataframe and export

In [ ]:
rows = []


for pid in pid_list:
    k_outline = get_num_keystrokes(pid, "outline")
    k_draft = get_num_keystrokes(pid, "draft")
    k_revision = get_num_keystrokes(pid, "revision")
    k_backspace = get_keystroke_events(pid, "backspace")

    rows.append({
        "participant_id": pid,
        "condition": get_condition(pid),
        "prompt_id": get_prompt_id(pid),
        "time_on_outline": get_time_on_task(pid, "outline"),
        "time_on_draft": get_time_on_task(pid, "draft"),
        "time_on_revision": get_time_on_task(pid, "revision"),
        "keystrokes_outline": k_outline,
        "keystrokes_draft": k_draft,
        "keystrokes_revision": k_revision,
        "wc_outline": len(get_essay(pid, "outline").split(" ")),
        "wc_draft": len(get_essay(pid, "draft").split(" ")),
        "wc_revision": len(get_essay(pid, "revision").split(" ")),
        "num_paste_events": get_keystroke_events(pid, "paste"),
        "num_backspace_events": k_backspace,
        "backspace_frac": k_backspace / (k_outline + k_draft + k_revision),
        "api_requests": get_api_requests(pid),
        "revision_edit_distance": get_edit_distance(pid)
    })

behavioral_df = pd.DataFrame(rows)
behavioral_df

In [ ]:
behavioral_df.to_csv('csv_exports/behavioraldata.csv', index=False)

# Export submitted text

Note that we're rewrite the get_essay function to return an empty string in the ai draft condition.

In [ ]:
def get_ai_draft(pid):
    filtered = ilogs.loc[ilogs["participant_id"] == pid]
    
    phrase = "Please write a complete draft essay based on this outline and prompt"
    matches = filtered[filtered["event_data"].apply(lambda d: isinstance(d, dict) and phrase in d.get("prompt", ""))]
    
    if len(matches) < 1:
        print(f"error [{pid}]: no ai draft found")
        return ""
    
    if len(matches) > 1:
        print(f"warning [{pid}]: more than one ai draft found")
        matches = matches.drop_duplicates(subset="participant_id", keep="last")
    
    return matches.iloc[0]["event_data"]["response"]

def get_essay(pid, stage):
    filtered = snapshots.loc[
        (snapshots["participant_id"] == pid) & 
        (snapshots["type"] == "final") &
        (snapshots["stage"] == stage)
    ]
    # if condition 3, need to pull draft from api request
    if len(filtered) == 0:
        condition = surveys.loc[surveys["participant_id"] == pid].iloc[0]["condition"]
        if condition == "3":
            return ""
        else:
            print(f"error [{pid}]: no final submission [{stage}]")
            return "error"
    if len(filtered) > 1:
        print(f"warning [{pid}]: more than one final submission [{stage}]")
        filtered = filtered.drop_duplicates(subset="participant_id", keep="last")
    
    return filtered.iloc[0]["text_content"]

In [ ]:
rows = []

for pid in pid_list:
    condition = get_condition(pid)
    if condition == "3":
        human_draft = ""
        ai_draft = get_ai_draft(pid).replace("\n", " ")
    else:
        human_draft = get_essay(pid, "draft").replace("\n", " ")
        ai_draft = ""
        
    rows.append({
        "participant_id": pid,
        "condition": get_condition(pid),
        "prompt_id": get_prompt_id(pid),
        "outline_text": get_essay(pid, "outline").replace("\n", " "), 
        "human_draft_text": human_draft, 
        "ai_draft_text": ai_draft, 
        "final_text": get_essay(pid, "revision").replace("\n", " ")
    })

text_df = pd.DataFrame(rows)
text_df

In [ ]:
text_df.to_csv('csv_exports/textdata.csv', index=False)

# Export AI support data

Note that values go to 0 if there is no AI support for a given condition.

- **outline_ideas_shown**: Sum of the number of "thesis" suggestions / ideas shown to the participant, counted via api requests. (Note that due to logging errors this is either 0 or 3; we don't know if they requested more than 3.)
- **outline_detail_shown**: Sum all times "generate outline" is requested.
- **outline_ideas_pasted**: Count number of times a paste event was of text that was in the AI response (i.e. an idea /outline from the sidebar).
- **outline_characters_inserted**: Number of characters pasted from the subset of paste events above.
- **revision_ideas_shown**: Number of revision suggestions shown (typically 1–4 suggestions returned per request).
- **revision_ideas_applied**: Count number of revision suggestions applied to text by checking final essay for suggested fixes.
- **revision_characters_inserted**: Number of characters in final submission that came from suggested fixes.

## Outline metrics

In [ ]:
def get_outline_ideas_shown(pid):
    # Count ideas shown based on which prompt was used in outline-stage API calls
    # note that the "generate a single idea" prompt wasn't logged :'(
    filtered = ilogs.loc[
        (ilogs["participant_id"] == pid) &
        (ilogs["stage"].str.lower() == "outline") &
        (ilogs["event_type"].str.contains("api_call:success", na=False))
    ]
    total = 0
    for _, row in filtered.iterrows():
        prompt = row["event_data"]["prompt"]
        pl = prompt.lower()
        if "please give me 3 distinct thesis ideas for this essay" in pl:
            total += 3
        elif "generate a single interesting thesis statement" in pl:
            total += 1
        elif "create a very concise outline for" in pl:
            continue
        elif "please write a complete draft essay based on this" in pl:
            continue
        else:
            print(f"unrecognized outline prompt for {pid}: {prompt[:100]!r}")
    return total


def get_outline_detail_shown(pid):
    # Count outline detail requests by checking the prompt used in outline-stage API calls
    filtered = ilogs.loc[
        (ilogs["participant_id"] == pid) &
        (ilogs["stage"].str.lower() == "outline") &
        (ilogs["event_type"].str.contains("api_call:success", na=False))
    ]
    count = 0
    for _, row in filtered.iterrows():
        prompt = row["event_data"]["prompt"]
        if "create a very concise outline for" in prompt.lower():
            count += 1
    return count


def check_if_from_recent_cut_copy(pid, paste_row, pasted_text):
    """Check if pasted text comes from recent cut or copy events"""
    
    # Get cut/copy events from the same stage and participant
    cut_copy_events = ilogs.loc[
        (ilogs["participant_id"] == pid) &
        (ilogs["stage"].str.lower() == paste_row["stage"]) &
        (ilogs["event_type"].isin(["keystroke:cut", "keystroke:copy"])) &
        (ilogs["time_from_stage_start"] <= paste_row["time_from_stage_start"])  # Only events before the paste
    ]
    
    # Check if any cut/copy event contains the pasted text
    for _, event_row in cut_copy_events.iterrows():
        if "text" in event_row["event_data"]:
            cut_copy_text = event_row["event_data"]["text"]
            if pasted_text in cut_copy_text or cut_copy_text in pasted_text:
                return True
    
    return False


def get_outline_ideas_pasted(pid):
    """returns num ideas pasted and total chars pasted"""
    # legacy code below; all pastes are from AI suggestions
#     calls = ilogs.loc[
#         (ilogs["participant_id"] == pid) &
#         (ilogs["stage"].str.lower() == "outline") &
#         (ilogs["event_type"].str.contains("api_call:success", na=False))
#     ]

#     responses = []
#     for _, row in calls.iterrows():
#         resp = row["event_data"]["response"]
#         responses.append(resp)
#     if not responses:
#         return 0, 0

    pastes = ilogs.loc[
        (ilogs["participant_id"] == pid) &
        (ilogs["stage"].str.lower() == "outline") &
        (ilogs["event_type"] == "keystroke:paste")
    ]

    applied = 0
    total_chars = 0
    for _, row in pastes.iterrows():
        pasted = row["event_data"]["text"]
        is_from_cut_copy = check_if_from_recent_cut_copy(pid, row, pasted)
        # Ignore trivial very short pastes to reduce false positives
        if len(pasted.strip()) < 5:
            continue
        applied += 1
        total_chars += len(pasted)
        # legacy code below; all pastes are from AI suggestions
#         if any(pasted in r for r in responses) or is_from_cut_copy:
#             applied += 1
#             total_chars += len(pasted)
#         else:
#             if get_condition(pid) == "2":
#                 print(f"\n\n***\n\npaste event [{pid}]:", pasted)
    return applied, total_chars

## Revision metrics

In [ ]:
import re

def get_revision_ideas_shown(pid):
    # Sum suggestions returned across tool calls by parsing LLM responses
    filtered = ilogs.loc[
        (ilogs["participant_id"] == pid) &
        (ilogs["stage"].str.lower() == "revision") &
        (ilogs["event_type"].str.contains("api_call:success", na=False))
    ]
    total = 0
    for _, row in filtered.iterrows():
        resp = row["event_data"]["response"]
        total += max(0, len(re.split(r"###\s*Issue\s+\d+", resp)) - 1)
    return total


def get_revision_ideas_applied(pid):
    # checks for revision ideas that end up in the final essay
    filtered = ilogs.loc[
        (ilogs["participant_id"] == pid) &
        (ilogs["stage"].str.lower() == "revision") &
        (ilogs["event_type"].str.contains("api_call:success", na=False))
    ]
    
    if len(filtered) == 0:
        return 0, 0 
    
    final_essay = get_essay(pid, "revision")
    
    applied = 0
    total_chars = 0
    for _, row in filtered.iterrows():
        resp = row["event_data"]["response"]
        for line in resp.split('\n'):
            if "Addition text" in line or "Fix" in line:
                suggestion = line.split(":")[1].strip().strip('"')
                if suggestion in final_essay:
                    applied += 1
                    total_chars += len(suggestion)
    return applied, total_chars

# deprecated

# def get_revision_ideas_pasted(pid):
#     """returns num ideas pasted and total chars pasted"""
#     calls = ilogs.loc[
#         (ilogs["participant_id"] == pid) &
#         (ilogs["stage"].str.lower() == "revision") &
#         (ilogs["event_type"].str.contains("api_call:success", na=False))
#     ]

#     responses = []
#     for _, row in calls.iterrows():
#         resp = row["event_data"]["response"]
#         responses.append(resp)
#     if not responses:
#         return 0, 0

#     pastes = ilogs.loc[
#         (ilogs["participant_id"] == pid) &
#         (ilogs["stage"].str.lower() == "revision") &
#         (ilogs["event_type"] == "keystroke:paste")
#     ]

#     applied = 0
#     total_chars = 0
#     for _, row in pastes.iterrows():
#         pasted = row["event_data"]["text"]
#         if len(pasted.strip()) < 5:
#             continue
#         if any(pasted in r for r in responses):
#             applied += 1
#             total_chars += len(pasted)

#     return applied, total_chars

In [ ]:
a = get_revision_ideas_shown("65cb7b14e78fc98ecd71d8de")
b = get_revision_ideas_applied("65cb7b14e78fc98ecd71d8de")

a, b

## Build dataframe and save to CSV

In [ ]:
# Build AI support dataframe for CSV export
rows = []

for pid in pid_list:
    
    condition = get_condition(pid)
    
    if condition == "2":
        outline_pasted, outline_chars = get_outline_ideas_pasted(pid)
    else:
        outline_pasted = 0
        outline_chars = 0
    
    if condition == "4":
        revision_applied, revision_chars = get_revision_ideas_applied(pid)
    else:
        revision_applied = 0
        revision_chars = 0
        
    rows.append({
        "participant_id": pid,
        "condition": condition,
        "prompt_id": get_prompt_id(pid),
        # Outline
        "outline_ideas_shown": get_outline_ideas_shown(pid),
        "outline_detail_shown": get_outline_detail_shown(pid),
        "outline_ideas_pasted": outline_pasted,
        "outline_characters_inserted": outline_chars,
        # Revision
        "revision_ideas_shown": get_revision_ideas_shown(pid),
        "revision_ideas_applied": revision_applied,
        "revision_characters_inserted_1": revision_chars,
    })

ai_support_df = pd.DataFrame(rows)
ai_support_df.head()

In [ ]:
pd.set_option('display.max_rows', None)
ai_support_df.loc[ai_support_df["condition"] == "2"]

In [ ]:
ai_support_df.to_csv('csv_exports/ai_support.csv', index=False)